In [1]:
import json
import os
import pandas as pd
import more_itertools as mit
from collections import Counter
import collections 
import pickle
import time
import copy
from itertools import combinations
from collections import OrderedDict    


In [2]:
all_files = ["Long_Batch_4364840_0_100_results_v3" ,
             "Long_Batch_4367993_100_300_results_v2" ,
             "Long_Batch_4371552_300_500_results_v3" ,
             "Long_Batch_4374976_500_700_results_v3" ,
             "Batch_4360660_100_200_results" 
             ,"Batch_4362039_200_500_results" ,
             "Batch_4363418_500_1100_results_v5" ,
             "Batch_4377561_1100_1600_results_v2" ,
             "Batch_4383845_1600_1900_results"]

amt_results = None
for filename in all_files:
    if amt_results is not None:
        read_in = pd.read_csv("amt_results/raw_csv/" + filename + ".csv")
        amt_results = pd.concat([amt_results, read_in], axis=0)
    else:
        amt_results = pd.read_csv("amt_results/raw_csv/" + filename + ".csv")
        
amt_results.shape  

(7612, 32)

In [3]:
run1="AMT_First_batch_03032021"
run2="AMT_2nd_batch_09032021"
run3="Batch_4360660_100_200_results"
run4="Batch_4362039_200_500_results"
run5="Batch_4363418_500_1100_results"
run6="Batch_4363418_500_1100_results_v2"
run7 = "Long_Batch_4364840_0_100_results"
run8 = "Long_Batch_4364840_0_100_results_v2"
run9 = "Long_Batch_4367993_100_300_results"
run10 = "Long_Batch_4371552_300_500_results"
run11 = "Long_Batch_4371552_300_500_results_v2"
run12 = "Long_Batch_4374976_500_700_results"
run13 = "Long_Batch_4374976_500_700_results_v2"
run14="Batch_4363418_500_1100_results_v5"
run15 = "Batch_4377561_1100_1600_results"
run16 = "Long_Batch_4364840_0_100_results_v3"
run17= "Long_Batch_4367993_100_300_results_v2"
run18= "Long_Batch_4371552_300_500_results_v3"
run19= "Long_Batch_4374976_500_700_results_v3"
run20= "Batch_4377561_1100_1600_results_v2"
run21="Batch_4383845_1600_1900_results"

#Means to read in single files..
#filename = run21
#amt_results = pd.read_csv("amt_results/raw_csv/" + filename + ".csv")

amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].astype('str')
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].astype('str') 
amt_results.dtypes

HITId                           object
HITTypeId                       object
Title                           object
Description                     object
Keywords                        object
Reward                          object
CreationTime                    object
MaxAssignments                   int64
RequesterAnnotation             object
AssignmentDurationInSeconds      int64
AutoApprovalDelayInSeconds       int64
Expiration                      object
NumberOfSimilarHITs            float64
LifetimeInSeconds              float64
AssignmentId                    object
WorkerId                        object
AssignmentStatus                object
AcceptTime                      object
SubmitTime                      object
AutoApprovalTime                object
ApprovalTime                    object
RejectionTime                   object
RequesterFeedback               object
WorkTimeInSeconds                int64
LifetimeApprovalRate            object
Last30DaysApprovalRate   

# Filter to only SUBMITTED IF NEEDED and remove rejected rows.

In [4]:
amt_results.shape

(7612, 32)

In [5]:
amt_results = amt_results[amt_results['RejectionTime'].isna()]
amt_results.shape

(7396, 32)

In [6]:
amt_results['AssignmentStatus'].value_counts()

Approved     6498
Submitted     898
Name: AssignmentStatus, dtype: int64

In [7]:
#amt_results = amt_results[amt_results['AssignmentStatus']=='Submitted'].copy()
amt_results.shape

(7396, 32)

# Convert HTML encoded characters back to normal

In [8]:
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.strip("'")
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.strip("'")

amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.replace('&#44', ',')
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.replace('&#39', '\'')
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.replace('&#34', '\"')
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.replace('&#44', ',')
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.replace('&#39', '\'')
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.replace('&#34', '\"')

# Majority vote Aggregation Mechanism



In [9]:
def calculate_annotated_ranges(taskAnswers):
    
    answer= json.loads(taskAnswers)
    marked_chars = []
    if len(answer) != 0 and  len(list(answer[0].keys())) != 0:
        key= list(answer[0].keys())[0] 
        for ent in answer[0][key]['entities']:
            #entity_tuples.append((ent['startOffset'], ent['endOffset']))
            marked_chars.extend(range(ent['startOffset'], ent['endOffset']))
    else:
        print("Empty annotation!!")
        
    return marked_chars
amt_results['marked_chars'] = amt_results['Answer.taskAnswers'].apply(lambda answer: calculate_annotated_ranges(answer))    

Empty annotation!!
Empty annotation!!
Empty annotation!!
Empty annotation!!
Empty annotation!!
Empty annotation!!


In [10]:
def aggregate_ranges(series):
    counter = collections.Counter() 
    for annotation in series:
        counter.update(Counter(annotation)) 

    threshold = len(series) / 2
    mv_dict= {key:val for key, val in dict(counter).items() if val >= threshold}
    final_ranges=[]
    
    for group in mit.consecutive_groups(list(mv_dict.keys())):
        group_list = list(group)
        final_ranges.append((group_list[0], group_list[-1]))
    return final_ranges

#test = amt_results[amt_results.HITId == '3BFF0DJK8XC0JVZWP5F1P36TQW6STQ']
#test.groupby('HITId').marked_chars.agg(aggregate_ranges)
mv_annotations = amt_results.groupby('HITId').marked_chars.agg(aggregate_ranges)

def convert_marked_chars_to_spans(marked_chars):
    final_ranges=[]
    for group in mit.consecutive_groups(marked_chars):
        group_list = list(group)
        final_ranges.append((group_list[0], group_list[-1]))
    return final_ranges
    


amt_results['marked_spans'] = amt_results['marked_chars'].apply(lambda marked_chars: convert_marked_chars_to_spans(marked_chars))

# Performance Evaluation
Create Worker dict
Loop over combination of workers --> evaluate 

In [11]:
worker_annotations = {}
def write_to_dict(worker_id, unit_id, anno_list):
    if worker_id in worker_annotations:
        current_elements = worker_annotations[worker_id]
        current_elements[unit_id] = anno_list
        worker_annotations[worker_id] = current_elements
    else:
        worker_annotations[worker_id] = {unit_id:anno_list}
    
    
amt_results.apply(lambda row: write_to_dict(row['WorkerId'], row['HITId'], row['marked_chars']), axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
893    None
894    None
895    None
896    None
897    None
Length: 7396, dtype: object

In [12]:
# Credits to Louis de Bruijn (https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3)
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

In [13]:
# FScore formula for inter annotator agreement from: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1090460/

def write_to_worker(worker_id, a, b, c):
    if worker_id in worker_a:
        worker_a[worker_id] = worker_a[worker_id] + a
        worker_b[worker_id] = worker_b[worker_id] + b
        worker_c[worker_id] = worker_c[worker_id] + c
    else:
        worker_a[worker_id] = a
        worker_b[worker_id] = b
        worker_c[worker_id] = c
    

micro_fscores={}
cohen_kappas = {}
unique_worker_combis = list(combinations(amt_results.WorkerId.unique(), 2))

#Global Fscore
g_a = 0
g_b = 0
g_c = 0

#Worker specific performance IAA
worker_a={}
worker_b={}
worker_c={}

for (worker1, worker2) in unique_worker_combis:
    intersection = worker_annotations[worker1].keys() & worker_annotations[worker2].keys()
    w1_full_annos = []
    w2_full_annos = []
    
    # Calculate Measures only for overlapping pairs..
    if len(intersection) > 0: 
        #micro Fscores
        a = 0
        b = 0
        c = 0
        for item in intersection:
            a += len(set(worker_annotations[worker1][item]) & set(worker_annotations[worker2][item]))
            b += len(set(worker_annotations[worker1][item]) - set(worker_annotations[worker2][item]))
            c += len(set(worker_annotations[worker2][item]) - set(worker_annotations[worker1][item]))
            
            #Create 0/1 list for whole description for cohen's kappa
            amt_row=amt_results[amt_results['HITId'] == item].iloc[0]
            if len(amt_row['Input.clean_description']) <= 0:
                print('ERROR at %s'%(item))
            worker1_indexes = [0] * len(amt_row['Input.clean_description'])
            worker2_indexes = [0] * len(amt_row['Input.clean_description'])
            for i in worker_annotations[worker1][item]:
                worker1_indexes[i] = 1 
            for i in worker_annotations[worker2][item]:
                worker2_indexes[i] = 1 
            w1_full_annos.extend(worker1_indexes)
            w2_full_annos.extend(worker2_indexes)
            
            #count for each worker indiviually
            write_to_worker(worker1, a, b, c)
            write_to_worker(worker2, a, b, c)
            
        g_a += a
        g_b += b
        g_c += c
            
        #print(item, ' a:', a, ' b:', b, 'c:', c)
        #F-Score for user pair
        if a != 0 or b != 0 or c != 0:
            fscore = 2*a /  ( (2*a) + b + c )
            micro_fscores[(worker1, worker2)] = fscore
            #Cohens Kappa for user pai
            cohen_kappas[(worker1, worker2)] = cohen_kappa(w1_full_annos, w2_full_annos)
        else:
            fscore = 1 #TODO Check if that is really true

            


In [14]:
#Report Global F1 Score        
global_fscore = 2*g_a /  ( (2*g_a) + g_b + g_c )
print('The Mikro F-Score is: ', global_fscore)

#Micro Average F1 Score
makro_fscore = 0
for val in micro_fscores.values(): 
    makro_fscore += val 

makro_fscore = makro_fscore / len(micro_fscores)
print('The Makro F-Score is: ', makro_fscore)

#Micro Averaged Cohens Kappa
makro_cohen = 0
for val in cohen_kappas.values(): 
    makro_cohen += val 

makro_cohen = makro_cohen / len(cohen_kappas) 
print('The Avg. Pairwise Cohen Kappa is: ', makro_cohen)

#micro_fscores

The Mikro F-Score is:  0.7908468510904161
The Makro F-Score is:  0.7755108807684655
The Avg. Pairwise Cohen Kappa is:  0.7603251655629136


In [15]:
#workerwise F1 Score / IAA
worker_f1 = {w_id:2*w_a /  ( (2*w_a) + worker_b[w_id] + worker_c[w_id] ) for (w_id, w_a) in worker_a.items()}
worker_f1

sorted(worker_f1.items())

[('A104V8NZIQFN2F', 0.7631517080152492),
 ('A12R2U6TBB3OOG', 0.9443507588532883),
 ('A1DD23J1WBGQUU', 0.8259109311740891),
 ('A1EZV3Q6PGMQTY', 0.7259073842302879),
 ('A1FGKIKJYSL1MI', 0.78475591378219),
 ('A1J8TVICSRC70W', 0.4689655172413793),
 ('A1NIO3JNX5QTN8', 0.18448637316561844),
 ('A1NZFJHVJ9CNTO', 0.7441860465116279),
 ('A1SN0UU3V61IIM', 0.8638648363252376),
 ('A22CRWMZUX7FFR', 0.905511811023622),
 ('A22KRF782ELLB0', 0.9364396654719236),
 ('A23WTODPHXZ68Y', 0.6125025502917534),
 ('A2460MAJX0NA82', 0.5368770764119601),
 ('A27SMEOPKV84VI', 0.8993036365646071),
 ('A29VL3MZE7YPBZ', 0.8255262071811803),
 ('A2AFHECSC7X9ZL', 0.8414814814814815),
 ('A2DDCDTHI8TBUT', 0.7154240182128628),
 ('A2DHLFPCDUTK2O', 0.8384071823256964),
 ('A2ESBX4C2Q6TQY', 0.7675675675675676),
 ('A2HM35CWB7IIFM', 0.9165892624930336),
 ('A2MCG5W6LHSRG9', 0.9895366218236173),
 ('A2QVMCGDLTWV9', 0.6870787780910985),
 ('A2T5CROJ0FWIJY', 0.7959540382049665),
 ('A2UF2FRGVW4T89', 0.7963184463762958),
 ('A2UIGDOLX5RV95',

# Checkout Low-IAA performance Workers

In [16]:
sorted( ((v,k) for k,v in micro_fscores.items()), reverse=False)

[(0.0, ('A1FGKIKJYSL1MI', 'A1NIO3JNX5QTN8')),
 (0.0, ('A23WTODPHXZ68Y', 'A2QVMCGDLTWV9')),
 (0.0, ('A2UIGDOLX5RV95', 'A104V8NZIQFN2F')),
 (0.0, ('A2UIGDOLX5RV95', 'A3BVPQFBYGWJWX')),
 (0.0, ('A2UIGDOLX5RV95', 'APXNY64HXO08K')),
 (0.0, ('A3696JXTRKL2FI', 'A2WNW8A4MOR7T7')),
 (0.0, ('A38NFX88VZDMJ3', 'A1NIO3JNX5QTN8')),
 (0.0, ('A38NFX88VZDMJ3', 'A3LRZX8477TYYZ')),
 (0.12087912087912088, ('A34QZDSTKZ3JO9', 'A1NIO3JNX5QTN8')),
 (0.15853658536585366, ('A29VL3MZE7YPBZ', 'A2UIGDOLX5RV95')),
 (0.18691588785046728, ('A2UIGDOLX5RV95', 'A27SMEOPKV84VI')),
 (0.26666666666666666, ('A2UF2FRGVW4T89', 'A2460MAJX0NA82')),
 (0.2682926829268293, ('APGX2WZ59OWDN', 'A2QVMCGDLTWV9')),
 (0.3157894736842105, ('A38NFX88VZDMJ3', 'A1J8TVICSRC70W')),
 (0.32, ('A2460MAJX0NA82', 'A1SN0UU3V61IIM')),
 (0.33962264150943394, ('A3U21PUMQ6NGT2', 'A9WGDBDXUNLOD')),
 (0.3546099290780142, ('A2UF2FRGVW4T89', 'A3696JXTRKL2FI')),
 (0.35555555555555557, ('A34QZDSTKZ3JO9', 'A2460MAJX0NA82')),
 (0.38493723849372385, ('A37WDOIQH6

# Convert to Labelstudio format

In [17]:
amt_results['dream_id'] = amt_results['Input.dream_id_name'].str.split(' - ', n=1).str[0]

 # Only one split --> all other patterns are by accident --> n=1
amt_results['name'] = amt_results['Input.dream_id_name'].str.split(' - ', n=1).str[1]

amt_results['name'].head()

0    "Crystal Candy" Purest and real MDMA 92% - 2 Gr -
1    "Crystal Candy" Purest and real MDMA 92% - 2 Gr -
2    "Crystal Candy" Purest and real MDMA 92% - 2 Gr -
3         10G Ketama Sputnik (Sativa) ~ ESCROW PRICE ~
4         10G Ketama Sputnik (Sativa) ~ ESCROW PRICE ~
Name: name, dtype: object

In [18]:
#Load data used for dataset creation
df_original = pd.read_pickle('intermediate_store/preprocessed_AMT_ads_V1.2.pickle')

In [19]:
with open('appen_results/label_studio_example.json') as json_file:
    label_studio = json.load(json_file)
label_studio[0]

{'completions': [{'created_at': 1610963709,
   'id': 641001,
   'lead_time': 11.385,
   'result': [{'from_name': 'label',
     'id': '9HDYDBGmSQ',
     'to_name': 'text',
     'type': 'labels',
     'value': {'end': 40,
      'labels': ['Drug'],
      'start': 30,
      'text': 'Lorazepam '}}]}],
 'data': {'Unnamed: 0': 11228,
  'category': 'DRUG',
  'description': 'You are purchasing one box of Lorazepam 5 mg with 60 pills packaged in flaskYou will receive exactly the box in the picture because I took the photo of the box I am sellingAll my medicines come from a pharmacy in Europe and are according to EMEA rules  ( European Medicines Agency )  . Our products are shipped in registered mail with tracking number and are carefully packaged .  For more information about the product please contact me',
  'keywords': "[('AA, Benzos', 30, 39)]",
  'lang': 'en',
  'market_name': 'DREAM',
  'name': 'Lorazepam 5 mg  60 pills  tavor 5 mg Ativan Temest',
  'subcategory': 'Drugs, Benzos',
  'vendor

In [20]:
df_original[df_original['dream_id'] == 274501]

,dream_id,vendor,name,description,lang,doc,clean_description
145,274501,Gecko,Custom MMXXX,Soma 350mg x 400 - $85\nAlbuterol Inhalers x 1...,en,"[\n [\n {\n ""id"": 1,\n ""text"": ""...",Soma 350 mg x 400 - 85 Albuterol Inhalers x 10...


In [21]:

def get_result_json(annotation_list, text, hitID):
    result=[]
    indx = 0           
    for (startIdx, endIdx) in annotation_list:
        anno_dict = {'from_name': 'label',
                       'id': str(indx) + '-' + hitID,
                       'to_name': 'text',
                       'type': 'labels',
                       'value':{'end':endIdx+1,
                                    'labels': ['Drug'],
                                    'text':text[startIdx:endIdx+1],
                                    'start':startIdx}}
        result.append(anno_dict)
        indx = indx + 1
    #print('result:', result)
    return result

def create_labelstudio_data(agg_row):           
    original_row = df_original[df_original['dream_id'] == int(agg_row['dream_id'])].iloc[0]
    mv_annotation = mv_annotations[agg_row['HITId']]
    data={'dream_id':agg_row.dream_id, 'name':agg_row['name'], 'p_id':int(original_row.name), 
          'clean_description':agg_row['Input.clean_description'],
         'WorkerId':agg_row['WorkerId'], 'worktime':agg_row['WorkTimeInSeconds'],
         'AssignmentID':agg_row['AssignmentId']}

    completions={'created_at': int(time.time()),
                'id':agg_row.HITId,
                'lead_time':10.00,        #I have no clue what lead time is.
                'result':get_result_json(agg_row['marked_spans'], agg_row['Input.clean_description'], agg_row.HITId)
                }
    
    return {'completions':[completions], 'data':data, 'id':agg_row.HITId}


amt_results['labelstudio_data'] = amt_results.apply(lambda row: create_labelstudio_data(row), axis=1)


In [22]:
#Write TO File
with open('amt_results/amt_final_ind/label_studio_' + filename + '_individual.json', 'w') as json_file:
    json.dump(amt_results['labelstudio_data'].tolist(), json_file, indent='\t')